<a href="https://colab.research.google.com/github/lisaadnr/ekstraksi-llama/blob/main/Extracted_Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

# Path ke folder yang berisi file JSON
folder_path = '/content/sample_data/input'

# Fungsi untuk mengekstrak entitas
def extract_entities(labels):
    # Mengambil string label dan memecahnya berdasarkan spasi
    label_string = labels['id']
    label_parts = label_string.split()

    # Mengambil hanya bagian entitas (sebelum '/') dan mengabaikan label '/O'
    entities = [part.split('/')[0] for part in label_parts if '/' in part and part.split('/')[1] != 'O']

    return entities

# Membaca setiap file JSON di folder dan memprosesnya
output = []

# Menelusuri semua file di dalam folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):  # Pastikan hanya file JSON yang diproses
        file_path = os.path.join(folder_path, filename)

        # Membaca data dari file JSON
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)

                # Memproses setiap verse dalam chapter
                for chapter, verses in data.items():
                    for verse_id, verse_data in verses.items():
                        # Ekstraksi entitas dari label
                        entities = extract_entities(verse_data['labels'])

                        # Menambahkan entitas ke output dalam format tuple untuk memudahkan pengurutan
                        for entity in entities:
                            output.append((int(verse_data['chapterid']), int(verse_data['verse_id']), entity))
            except json.JSONDecodeError:
                print(f"File {filename} tidak dapat dibaca sebagai JSON.")

# Mengurutkan output berdasarkan chapterid dan verse_id
output.sort(key=lambda x: (x[0], x[1]))  # Urutkan berdasarkan chapterid dan verse_id

# Menulis hasil output ke file TSV
output_file = '/content/sample_data/output2_sorted.tsv'
with open(output_file, 'w', encoding='utf-8') as f:
    # Menulis header
    f.write("chapter_id\tverse_id\tentitas\n")

    # Menulis data entitas yang sudah diurutkan
    for line in output:
        f.write(f"{line[0]}\t{line[1]}\t{line[2]}\n")

print(f"Output telah disimpan dalam file: {output_file}")


Output telah disimpan dalam file: /content/sample_data/output2_sorted.tsv


In [ ]:
import csv

# Path ke file input (output sebelumnya) dan output baru
input_file = '/content/sample_data/output2_sorted.tsv'
output_file = '/content/sample_data/output_cleaned.tsv'

# Fungsi untuk membersihkan tanda kurung di awal entitas
def clean_entity(entity):
    # Hapus tanda kurung di awal entitas
    if entity.startswith('('):
        return entity[1:]  # Menghapus karakter pertama yaitu '('
    return entity

# Membaca file input dan memproses entitas
cleaned_output = []

with open(input_file, 'r', encoding='utf-8') as infile:
    reader = csv.reader(infile, delimiter='\t')

    # Skip header
    header = next(reader)

    # Menambahkan header ke hasil yang sudah dibersihkan
    cleaned_output.append(header)

    # Membaca baris satu per satu, membersihkan entitas, dan menambahkannya ke list
    for row in reader:
        chapter_id = row[0]
        verse_id = row[1]
        entity = row[2]

        # Menghapus tanda kurung di awal entitas
        cleaned_entity = clean_entity(entity)

        # Menambahkan hasil yang sudah dibersihkan
        cleaned_output.append([chapter_id, verse_id, cleaned_entity])

# Menulis hasil yang sudah dibersihkan ke file output baru
with open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    writer = csv.writer(outfile, delimiter='\t')
    # Menulis hasil ke file
    writer.writerows(cleaned_output)

print(f"Output yang sudah dibersihkan disimpan dalam file: {output_file}")


Output yang sudah dibersihkan disimpan dalam file: /content/sample_data/output_cleaned.tsv
